In [ ]:
import nltk
from nltk import load_parser
# import grammar_selector #la idea es importar la función del otro script.

In [ ]:
# Esto es para testear una oración cualquiera usando de gramática integral grammar
sents = ['vos jugás']
grammar = 'integral_grammar.fcfg'
for results in nltk.interpret_sents(sents, grammar):
    for (synrep, semrep) in results:
             print(synrep)

In [ ]:
# La idea es que esto vaya a parar a grammar_selector.py


import nltk
from nltk.tokenize import word_tokenize
from nltk import load_parser
import re
import os

#############################
# Chequeo de Léxico
#############################

# Esto es para que al momento de importar este archivo reuna en un solo 
# archivo todo el léxico. 
# Si el archivo ya existe no lo vuelve a armar, trabaja con el que está.
# Estaría bueno repensar esto como una función para poder setear qué archivos
# uno quiere que coleccione, para de esa manera hacerlo más portable por ejemplo
# para otras lenguas.

if not os.path.exists('./reglas_automaticas/integral_lexicon.fcfg'):
    lexicon_files = os.listdir(path='./reglas_automaticas')
    outfile = open('./reglas_automaticas/integral_lexicon.fcfg', 'w')
    for fname in lexicon_files:
        with open(f'./reglas_automaticas/{fname}', 'r') as infile:
            for line in infile:
               outfile.write(line)
            
#############################
# Función para parsear 
#############################

# Esta sería la función que arma la gramática personalizada para cada oración
# En principio la pensé así (pero metan mano tranca):
# grammar_selector toma tres argumentos: una gramática de base, un léxico y una oración.
# 1) lo primero que hace es borrar integral_grammar.fcfg (la idea es usar siempre ese archivo
#   una y otra vez cada vez que se parsea. 
# 2) Después crea integral_grammar.fcfg de vuelta y le escribe todas las reglas de la gramática base
# 3) Después tokeniza la oración.
# 4) Por cada palabra de la lista de tokens, la idea sería que si esa palabra está al final de línea
#    entre comillas simples después de una flecha (no sé cuánto puede ser necesario especificar del 
#    contexto), agregue esa línea a integral_grammar
# 5) Finalmente, parsea la oración usando como base integral_grammar.fcfg (o sea, la gramática de base
#    más las reglas de reescritura que reescriben a los tokens que posee la oración).
# 
# En principio, entiendo que el punto 1, 2, 3 y 5 están bien, pero tendrían que chequear bien el 4.

def grammar_selector(grammar_base, lexicon, sentence):
# punto 1
    if os.path.exists('integral_grammar.fcfg'):
        os.remove('integral_grammar.fcfg')
# punto 2
    with open('integral_grammar.fcfg', 'w') as f:
        for line in grammar_base:
            f.write(line)
            #print(line)
# punto 3
    tokens = word_tokenize(sentence)
    #print(tokens)
# punto 4
    for word in tokens:
        for line in lexicon:
            #print(word)
            if line.__contains__(word): # con esto básicamente toma a word en cualquier lugar de la línea
                # esto genera problemas porque para 'el chico', por ejemplo, selecciona cualquier línea en 
                # la que haya un 'el', como la que reescribe "abellaca" por ejempo.
                with open("integral_grammar.fcfg", "a") as f:
                    f.write(line)
                    print(line)
# punto 5
    for results in nltk.interpret_sents(sentence, open("integral_grammar.fcfg", "r")):
        for (synrep, semrep) in results:
            print(synrep)

In [ ]:
# Para testar grammar_selector

integral_lexicon = open('reglas_automaticas/integral_lexicon.fcfg','r')
base_grammar = open('GramaticaDeRasgosBase.txt', 'r')
sent = "el chico vino"
grammar_selector(base_grammar, integral_lexicon, sent)